In [1]:

from s3_athena_api import *

env = "production"
site = 'a40ba08c-5e27-40d9-b798-7020ca6e4cae'
band = "5"
download_file = get_ap_scan(env, site, band)

s3_output= s3://mist-production-athena/results/
download_local_path = ../../downloads/
SELECT * FROM "secorapp_production"."cv_ap_scans" where site='a40ba08c-5e27-40d9-b798-7020ca6e4cae' and band=5 and dt='2020-03-20';
    
{'QueryExecutionId': '0f5a469e-ff07-4819-aeaf-5a34ed2743d4', 'ResponseMetadata': {'RequestId': '07a2b6a3-d3cf-40dc-9d6e-86b20b626e5f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 20 Mar 2020 23:36:22 GMT', 'x-amzn-requestid': '07a2b6a3-d3cf-40dc-9d6e-86b20b626e5f', 'content-length': '59', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
results/0f5a469e-ff07-4819-aeaf-5a34ed2743d4.csv ../../downloads/0f5a469e-ff07-4819-aeaf-5a34ed2743d4.csv
checking if key exist
checking if key exist
{'ResponseMetadata': {'RequestId': '5CC6EDEFC2741603', 'HostId': 'Ynb07BTMgiQkqM4VK5azvNomEWwBqZnmUubcZ5rtnIpMVfiPnsKqGYjQ68Y+b4iuyRcAb4RoDKY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Ynb07BTMgiQkqM4VK5azvNomEWwBqZnm

In [2]:
import sys
import os
import time
import boto3
from botocore.exceptions import ClientError


import datetime

def run_query(query, database, s3_output, region_name='us-east-1'):
    client = boto3.client('athena', region_name=region_name)
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': s3_output
        }
    )
    # print('Execution ID: ' + response['QueryExecutionId'])
    print(response)
    return response

def download_s3_file(response, download_path, region_name='us-east-1'):
    download_filename= download_path
    if response.get('ResponseMetadata').get('HTTPStatusCode') == 200:
        id = response.get('QueryExecutionId')
        file = 'results/' + id + '.csv'
        download_filename= download_path + id + '.csv'
        print(file, download_filename)
        while True:
            client = boto3.client('s3', region_name=region_name)
            s3_output_response = ""
            try:
                s3_output_response = client.get_object(Bucket=s3_bucket, Key=file)
                print(s3_output_response)
                client.download_file(Bucket=bucket, Key=file, Filename=download_filename)
                break
            except ClientError:
                print('checking if key exist')
            time.sleep(15)
    return download_filename

In [3]:

today = datetime.datetime.today().strftime("%Y-%m-%d")

env = 'staging'
env = "production"

s3_bucket = 'mist-{ENV}-athena'.format(ENV=env)
database = 'secorapp_{ENV}'.format(ENV=env)
table = 'cv_ap_scans'


s3_output = 's3://%s/results/' % s3_bucket
print("s3_output=",s3_output)
download_path = os.curdir + './../downloads/'
if not os.path.isdir(download_path):
    os.mkdir(download_path)

print("download_path = {}".format(download_path))


site='a799e12a-fc28-4ce7-a563-c2dd6d498e2f '
site = 'a40ba08c-5e27-40d9-b798-7020ca6e4cae'

# #SELECT * FROM "secorapp_production"."cv_ap_scans" where site='d1ee1d22-4b55-4c97-97c4-9d757144f45b' and ap2!='' and band=5 and tim > #
#CAST('2020-03-10 01:00:00' as timestamp) and tim < CAST('2020-03-11 18:00:00' as timestamp) and dt='2020-03-10' order by tim asc, ap2;

# query = '''SELECT date_trunc('hour', time) as time, org, site, ap, band, channel, ap2,
#  count(ap2) as counts, max(rssi) as max_rssi, max(in_site) as in_site
#  FROM "secorapp-{ENV}"."{TABLE}"
#  WHERE site="{SITE}" and time > date_add('day',-1,now())
#  GROUP BY time,org,site,ap,band,ap2,channel;'''.format(ENV=env,TABLE=table,SITE=site)

band='5'
query = '''SELECT * FROM "secorapp_{ENV}"."{TABLE}" where site='{SITE}' and band={BAND} and dt='{today}';
'''.format(ENV=env,TABLE=table,SITE=site, BAND=band, today=today)
print(query)

response = run_query(query, database, s3_output)
download_filename = download_s3_file(response, download_path)

print("download file size: ", os.path.getsize(download_filename))


s3_output= s3://mist-production-athena/results/
download_path = ../../downloads/
SELECT * FROM "secorapp_production"."cv_ap_scans" where site='a40ba08c-5e27-40d9-b798-7020ca6e4cae' and band=5 and dt='2020-03-20';

{'QueryExecutionId': '904a3442-be1c-45f3-814d-88e48d36a364', 'ResponseMetadata': {'RequestId': '7b3554e2-3e08-49f7-b20d-7160bb7b520f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 20 Mar 2020 23:37:07 GMT', 'x-amzn-requestid': '7b3554e2-3e08-49f7-b20d-7160bb7b520f', 'content-length': '59', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
results/904a3442-be1c-45f3-814d-88e48d36a364.csv ../../downloads/904a3442-be1c-45f3-814d-88e48d36a364.csv
checking if key exist
{'ResponseMetadata': {'RequestId': 'BA2BAF4F45C8903F', 'HostId': '510hrey34PVM0nRd43gBGtUliEEGBPYol6Lu6430wZxg9116uKe5yketH2v662PsN5VJ12EUTiI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '510hrey34PVM0nRd43gBGtUliEEGBPYol6Lu6430wZxg9116uKe5yketH2v662Ps

NameError: name 'bucket' is not defined

In [ ]:

#filename = download_path+file

In [ ]:
import pandas as pd

# filename = "../downloads/results/f92186ad-642b-4ec7-9d3b-7cd4aa33c492.csv"
df = pd.read_csv(download_filename)
df.head(10)

In [ ]:
# import csv

# with open(download_filename) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     line_count = 0
#     for row in csv_reader:
#         if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
#             line_count += 1
#         else:
#             if line_count< 10:
#                  print(row)
#             line_count += 1
#     print(f'Processed {line_count} lines.')